In [2]:
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError

C:\Users\yashs\AppData\Local\Temp/ipykernel_1860/2927579722.py:1: DeprecationWarning: Please use `QhullError` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  from scipy.spatial.qhull import QhullError


In [1]:
from xml.dom.minidom import parse
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
import pickle
from scipy.spatial.distance import cdist
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import sklearn
from skimage.transform import resize
from skimage.feature import hog
import pickle

In [12]:
# Define HOG parameters
winSize = (256, 256)
blockSize = (32, 32)
blockStride = (16, 16)
cellSize = (16, 16)
nbins = 9
orientations = 9

In [12]:
!mkdir -p /kaggle/working/dataset/helmet
!mkdir -p /kaggle/working/dataset/non-helmet

In [13]:
def extract_xml_data(file_path, xml_filename):
    label_path = file_path + '/' + xml_filename + '.xml'
    img_path = '/kaggle/input/hard-hat-detection/images' + '/' + xml_filename + '.png'
    domOBJ = parse(label_path)
    root = domOBJ.documentElement
    objects = root.getElementsByTagName("object")

    # for each identified and labelled object in the image, sort it and save it
    for idx, obj in enumerate(objects):
        cls_name = obj.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(obj.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(obj.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(obj.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(obj.getElementsByTagName("ymax")[0].childNodes[0].data)
        image = cv2.imread(img_path)
#         plt.imshow(image)
        cropped_image = image[y1:y2, x1:x2]
        try:
            cropped_image = cv2.resize(cropped_image, winSize)
        except:
            continue
#         plt.imshow(cropped_image)
        if cls_name == 'helmet':
            cv2.imwrite('/kaggle/working/dataset/helmet/' + xml_filename + f'_{idx}.png', cropped_image)
        else:
            cv2.imwrite('/kaggle/working/dataset/non-helmet/' + xml_filename + f'_{idx}.png', cropped_image)

In [14]:
files = os.listdir('/kaggle/input/hard-hat-detection/annotations')
files = files[:1000]
for file in files:
    file_xml = file.split(".")
    extract_xml_data('/kaggle/input/hard-hat-detection/annotations', file_xml[0])

In [15]:
label2id = {'non-helmet':1, 'helmet': 0}

In [16]:
HOG_Features = []
labels = []
datasetPath = '/kaggle/working/dataset/'
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

for label in list(label2id.keys()):
    print(label)
    if label == 'helmet':
        images = os.listdir(datasetPath + label + '/')
        for image in images[:len(labels)]:
            img = cv2.imread(datasetPath + label + '/' + image)
            features = hog.compute(img)
            HOG_Features.append(features.flatten())
            labels.append(label2id[label])
    else:
        images = os.listdir(datasetPath + label + '/')
        for image in images:
            img = cv2.imread(datasetPath + label + '/' + image)
            features = hog.compute(img)
            HOG_Features.append(features.flatten())
            labels.append(label2id[label])
    
HOG_Features = np.array(HOG_Features, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

non-helmet
helmet


In [17]:
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 1220, 1: 1220}


In [18]:
X_train, X_test, y_train, y_test = train_test_split(HOG_Features, labels, test_size=0.2, random_state=1789, shuffle=True)

In [19]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [20]:
print("score on training set params: ", clf.score(X_train, y_train))

score on training set params:  0.9866803278688525


In [21]:
print("score on testing set params: ", clf.score(X_test, y_test))

score on testing set params:  0.7540983606557377


In [22]:
filename = 'svm_model.yml'
pickle.dump(clf, open(filename, 'wb'))

In [3]:
# load the model
clf = pickle.load(open('./svm_classifier.pkl', 'rb'))

C:\Users\yashs\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\yashs\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\yashs\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


## Uploading a test image

### Without Helmet test case

In [4]:
testImage = cv2.imread('D:\My_Stuff\VIT-20BCE1789\Sem 8\Capstone\Datasets\Hard Hat\Hard Hat\workers\yash_seth.jpg')

In [5]:
testImage.shape

(666, 1000, 3)

### Apply Sliding window with different scales to get potential bounding boxes - apply classifier to those boxes to check if helmet or not

In [6]:
def sliding_window(image, stepSize, windowSize):
    windows = []
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            windows.append((x, y, image[y:y + windowSize[1], x:x + windowSize[0]]))
    return windows

In [8]:
def checkForHelmet(hog, window):
    if len(window) > 2:
        try:
            features = hog.compute(window[2])
        except:
            return False
        if len(features) != 0:
            features = features.flatten()
            try:
                pred = clf.predict([features])
            except:
                return False
    #         print("Pred:", pred)
            if pred[0] == 0:
                return True
            else:
                return False
        else:
            return False
    else:
        return False

In [10]:
windows = sliding_window(testImage, 64, [256, 256])
print(len(windows))

176


In [13]:
foundCandidates = []
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
for window in windows:
    if checkForHelmet(hog, window):
        foundCandidates.append(window[2])

: 

In [18]:
# _, axs = plt.subplots(len(foundCandidates) // 4, 4, figsize=(15, 15))
# axs = axs.flatten()
# # windowImg = []
# # for window in windows[:36]:
# #     windowImg.append(window[2])
    
# for img, ax in zip(foundCandidates, axs):
#     ax.imshow(img)
    
len(foundCandidates)

672

### With helmet test image

In [ ]:
testImage = cv2.imread('/kaggle/input/helmet-test-images/jimit_shah.jpg')

In [ ]:
testImage.shape

In [ ]:
windows = sliding_window(testImage, 32, [256, 256], hog)
print(len(windows))

In [ ]:
foundCandidates = []
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
for window in windows:
    if checkForHelmet(testImage, hog, window):
        foundCandidates.append(window[2])

In [ ]:
_, axs = plt.subplots(len(foundCandidates) // 6, 6, figsize=(8, 8))
axs = axs.flatten()
# windowImg = []
# for window in windows[:36]:
#     windowImg.append(window[2])
    
for img, ax in zip(foundCandidates, axs):
    ax.imshow(img)
    
plt.show()

In [ ]:
# image_height = 64
# image_width = 64
# window_size = 32
# window_step = 16

# def sliding_hog_windows(image):
#     hog_vector = []
#     for y in xrange(0, image_height, window_step):
#         for x in xrange(0, image_width, window_step):
#             window = image[y:y+window_size, x:x+window_size]
#             hog_vector.extend(hog(window, orientations=8, pixels_per_cell=(8, 8),
#                                             cells_per_block=(1, 1), visualise=False))
#     return hog_vector
    
# hog_vector = sliding_hog_windows(testImage)
# print(hog_vector)